## Label compilation pipeline

- This notebook will provide the functionality to generate and compile different labels into file(s)

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from pathlib import Path
import pandas as pd
from PIL import Image
import itertools
import torch
import os

In [ ]:
def get_model(model_name, device='cuda'):
    processor = Blip2Processor.from_pretrained(model_name)
    model = Blip2ForConditionalGeneration.from_pretrained(
                            model_name, 
                            ).to(device)
    return model, processor


def get_file_names(path, extension='.jpg'): 
    return [file_name for file_name in os.listdir(path) if file_name[-4:] == '.png']


def get_img_paths(folder_path, img_names):
     return [str(folder_path) + '/' + img_name for img_name in img_names]
    

def get_imgs(img_paths): return [Image.open(path) for path in img_paths]


def get_lbls(images, processor, model, device=None):
    if device is None: device = 'cuda' if torch.cuda.is_available() else 'cpu'
    inputs = processor(images=images, return_tensors='pt').to(device, torch.float32)
    generated_ids = model.generate(**inputs)
    return processor.batch_decode(generated_ids, skip_special_tokens=True)

def get_batch_idxs(img_paths, batch_size=10):
    idxs = [x for x in range(0, len(img_paths)+1, batch_size)]
    if len(img_paths) % batch_size != 0:
        last_idx = len(img_paths) - 1
        idxs.append(last_idx + 1)
    idx_tuples = [(idxs[x-1], idxs[x]) for x in range(1, len(idxs))]
    return idx_tuples

def get_batch(img_paths, idx_tup): 
        return get_imgs(img_paths[idx_tup[0] : idx_tup[1]])
    

def run_inf(img_paths, processor, model, device=None, batch_size=10):
    if device is None: device = 'cuda' if torch.cuda.is_available() else 'cpu'
    idxs = get_batch_idxs(img_paths)
    return list(itertools.chain(*[get_lbls(get_batch(img_paths, idx_tup), processor, model, device) \
                                  for idx_tup in get_batch_idxs(img_paths)]))

In [ ]:
img_folder = Path('data/campaign2/0021')
img_names = get_file_names(img_folder)
img_paths = get_img_paths(img_folder, img_names)
img_paths[:3]

['data/campaign2/0021/image_0021835.png',
 'data/campaign2/0021/image_0021801.png',
 'data/campaign2/0021/background_0021520.png']

In [ ]:
model_name = 'Salesforce/blip2-flan-t5-xl'
model, processor = get_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#list(model.named_children())[1]

In [ ]:
#[x for x in model.named_children()]

In [ ]:

#model_name = 'Salesforce/blip2-flan-t5-xl-coco'
#model_name = 'Salesforce/blip2-opt-2.7b'
#device = 'cuda'
#model, processor = get_model(model_name, device=device)

In [ ]:
#lbls = run_inf(img_paths, processor,model)
#len(lbls), lbls[:3]

In [ ]:
#model_base = model_name.split('/')[1]
#model_base

In [ ]:
#pd.DataFrame(zip(img_paths[:73], lbls), columns=['img_path', 'label'] \
#            ).to_csv(f'{model_base}.csv', index=False)

In [ ]:
#pd.read_csv(model_base + '.csv').head(3)

Looks like it works! lets wrap all of this into a single function and start compiling some labels!

* the models are HUGE, so will have to restart notebook between model runs *

In [ ]:
def compile_lbls(img_paths, model_name, batch_size=10):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model, processor = get_model(model_name, device=device)
    model_base = model_name.split('/')[1]
    lbls = run_inf(img_paths, processor, model, device, batch_size=25)
    pd.DataFrame(zip(img_paths, lbls), columns=['img_path', 'label'] \
            ).to_csv(f'{model_base}.csv', index=False)

In [ ]:
model_name = 'Salesforce/blip2-flan-t5-xl'
#model_name = 'Salesforce/blip2-opt-2.7b'
compile_lbls(img_paths, model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/azaidi/anaconda3/envs/dl/lib/python3.10/site-packages/transformers/generation/utils.py:1284: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
model_base = model_name.split('/')[1]
df = pd.read_csv(model_base + '.csv')
df.head(3)

,img_path,label
0,data/campaign2/0021/image_0021835.png,a green cylinder sitting on a dirt road
1,data/campaign2/0021/image_0021801.png,a green pipe is sitting on the ground in a dir...
2,data/campaign2/0021/background_0021520.png,a wooden planter with plants in it


In [ ]:
df.shape

(1100, 2)

## TO-DO:

- lets add nbdev/quatro support!
- ^this will alllow for cleaner code base and for running things from the terminal vs in notebooks --> which will allow for multiple runs back to back vs having to restart notebook bc of memory being held

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()